In [1]:
import os
import re
import nrrd

from ioutil import sample_paths

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

sns.set()
%matplotlib inline

In [2]:
def column_regexes(X):
    
    col_regexes = []
    for col in X.columns:
        items = col.split('_')
        if 'bins' in items[-1]:
            col_regexes.append(('_').join(items[1:-1]))
        else:
            col_regexes.append(('_').join(items[1:]))

    return np.unique(col_regexes)

In [ ]:
def prep_labels(labels, pref='CT'):
    
    new_labels = []
    for label in labels:

        items = label.split('_')

        if len(items) == 3:
            if items[1] == 'firstorder':
                kind = 'First Order'
            else:
                kind = items[1].upper()
                
            if len(items[2]) > 20:
                new_labels.append(f'{pref} {kind}\n{items[2]}')
            else:
                new_labels.append(f'{pref} {kind} {items[2]}')

        else:
            raise ValueError(f'Cannot handle label {label}')
    
    return new_labels

In [ ]:
def to_feature_categories(labels):
    """Process raw feature labels."""
    prep_labels = []
    for label in labels:
        if 'shape' in label:
            prep_labels.append('Shape')
        elif 'PETparam' in label:
            prep_labels.append('PET Parameter')
        elif 'firstorder' in label:
            prep_labels.append('First Order')
        elif 'glcm' in label:
            prep_labels.append('GLCM')
        elif 'gldm' in label:
            prep_labels.append('GLDM')
        elif 'glrlm' in label:
            prep_labels.append('GLRLM')
        elif 'glszm' in label:
            prep_labels.append('GLSZM')
        elif 'ngtdm' in label:
            prep_labels.append('NGTDM')
        else:
            prep_labels.append('Clinical')
    return prep_labels

In [3]:
def icc(Y):
    """Calculate intra-class correlation coefficient (ICC).

    Reference:
        Shrout, P. E., & Fleiss, J. L. (1979). Intraclass correlations: uses in
        assessing rater reliability. Psychological bulletin, 86(2), 420.

    Args:
        X (array-like): Data matrix with observations on rows
            and measurements on columns.

    Returns:
        (float): Intraclass correlation coefficient.

    """
    n, k = np.shape(Y)

    # Degrees of Freedom
    dfc = k - 1
    dfe = (n - 1) * (k-1)
    dfr = n - 1

    # Sum Square Total
    Y_avg = np.mean(Y)
    SST = np.sum((Y - Y_avg) ** 2)

    # Create the design matrix for the different levels:
    # * Sessions:
    x = np.kron(np.eye(k), np.ones((n, 1)))
    # * Subjects:
    x0 = np.tile(np.eye(n), (k, 1))
    X = np.hstack([x, x0])

    # Sum Square Error
    predicted_Y = np.dot(
        np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))), X.T), Y.flatten('F')
    )
    residuals = Y.flatten('F') - predicted_Y
    SSE = np.sum(residuals ** 2)

    MSE = SSE / dfe

    # Sum square column effect - between colums
    SSC = np.sum((np.mean(Y, axis=0) - Y_avg) ** 2) * n
    MSC = SSC / dfc / n

    # Sum Square subject effect - between rows/subjects
    SSR = SST - SSC - SSE
    MSR = SSR / dfr

    # ICC(3,1) = (mean square subject - mean square error) /
    # (mean square subject + (k-1)*mean square error)
    ICC = (MSR - MSE) / (MSR + (k-1) * MSE)

    return ICC

In [4]:
# Ng: Number of image intensities.
hassan_gl_transforms = {
    'original_glcm_DifferenceEntropy': lambda Ng, feature: feature / np.log(Ng ** 2),
    'original_glcm_JointEntropy': lambda Ng, feature: feature / np.log(Ng ** 2),
    'original_glcm_SumEntropy': lambda Ng, feature: feature * Ng,
    'original_glcm_Contrast': lambda Ng, feature: feature / (Ng ** 2),
    'original_glcm_DifferenceVariance': lambda Ng, feature: feature / (Ng ** 2),
    'original_glcm_SumAverage': lambda Ng, feature: feature / Ng,
    'original_glcm_DifferenceAverage': lambda Ng, feature: feature / Ng,
    'original_glrlm_GrayLevelNonUniformity': lambda Ng, feature: feature * Ng,
    'original_glrlm_HighGrayLevelRunEmphasis': lambda Ng, feature: feature / (Ng ** 2),
    'original_glrlm_ShortRunHighGrayLevelEmphasis': lambda Ng, feature: feature / (Ng ** 2),
    'original_ngtdm_Contrast': lambda Ng, feature: feature / Ng,
    'original_ngtdm_Complexity': lambda Ng, feature: feature / (Ng ** 3),
    'original_ngtdm_Strength': lambda Ng, feature: feature / (Ng ** 2),
}

In [5]:
# Nv: Number of voxels in ROI.
hassan_roi_transforms = {
    'original_firstorder_Energy': lambda Nv, feature: feature * Nv,
    'original_firstorder_Entropy': lambda Nv, feature: feature / np.log(Nv),
    'original_firstorder_TotalEnergy': lambda Nv, feature: feature / Nv,
    'original_glcm_Contrast': lambda Nv, feature: feature / Nv,
    'original_glcm_InverseVariance': lambda Nv, feature: feature / Nv,
    'original_glcm_JointAverage': lambda Nv, feature: feature / Nv,
    'original_glrlm_GrayLevelNonUniformity': lambda Nv, feature: feature / Nv,
    'original_ngtdm_Coarsness': lambda Nv, feature: feature * Nv,
    'original_ngtdm_Strength': lambda Nv, feature: feature * Nv,
}

In [6]:
def icc_from_hassan_modified(X, gl_bins):
    """Apply Hassan transform and record ICC for original and
    transformed features.

    X (pandas.DataFrame):
    gl_bins (array-like):

    Returns:
        (pandas.DataFrame):

    """
    icc_orig_feat = np.zeros(len(hassan_gl_transforms.keys()))
    icc_norm_feat = np.zeros(len(hassan_gl_transforms.keys()))
    for num, (key, transform) in enumerate(hassan_gl_transforms.items()):

        feats = X.filter(regex=key)
        #print(feats.head())
        X_transf = np.zeros_like(feats)
        for bin_num, (col, nbins) in enumerate(zip(feats.columns, gl_bins)):
            X_transf[:, bin_num] = transform(nbins, feats.loc[:, col].values)

        icc_orig_feat[num] = icc(feats.values)
        icc_norm_feat[num] = icc(X_transf)

    df_icc_orig_feat = pd.DataFrame(
        icc_orig_feat, index=hassan_gl_transforms.keys(), columns=['Score']
    )
    df_icc_orig_feat['Kind'] = ['Original'] * len(hassan_gl_transforms.keys())

    df_icc_norm_feat = pd.DataFrame(
        icc_norm_feat, index=hassan_gl_transforms.keys(), columns=['Score']
    )
    df_icc_norm_feat['Kind'] = ['Modified'] * len(hassan_gl_transforms.keys())
    df_icc = pd.concat((df_icc_orig_feat, df_icc_norm_feat), axis=0)

    df_icc.sort_values(by=['Score'])

    return df_icc

# SETUP

In [ ]:
ICC_THRESH = 0.8

In [ ]:
# Bin widths Z-scored CT stacks: Original images.
width32 = 0.1351010101010101
width64 = 0.06755050505050506
width128 = 0.03377525252525253

path_to_ct = './../../data_source/images/ct_nrrd/'
path_to_ct_masks = './../../data_source/images/masks_nrrd/'
path_to_features = './../../data_source/to_analysis/original_images/all_features_original_images.csv'
path_to_target = './../../data_source/to_analysis/original_images/dfs_original_images.csv'
path_hassan_icc_ct = './../../figures/compressing_feature_space/orig_images_hassan_icc_ct_texture.png'
path_hassan_icc_pet = './../../figures/compressing_feature_space/orig_images_hassan_icc_pet_texture.png'
path_to_icc = './../../data_source/to_analysis/compressed_features/all_features_orig_images_icc.csv'
path_scc_hassan_dropped =  './../../figures/compressing_feature_space/orig_images_scc_hassan_dropped_radiom_feats.png'
path_scc_hassan = './../../figures/compressing_feature_space/orig_images_scc_hassan_radiom_feats.png'
path_red_dropped = './../../data_source/to_analysis/compressed_features/all_features_orig_images_icc_dropped.csv'
path_pet_params_corr = './../../figures/compressing_feature_space/orig_images_pet_params_corr.png'


# Get number of GL

In [ ]:
path_ct_stacks = sample_paths(path_to_ct, path_to_ct_masks, target_format='nrrd')

gl_32bins = np.zeros(len(path_ct_stacks))
gl_64bins = np.zeros(len(path_ct_stacks))
gl_128bins = np.zeros(len(path_ct_stacks))

idx = []
for num, ct_path in enumerate(path_ct_stacks):
    
    fname = os.path.basename(ct_path['Image'])
    idx_num = re.findall(r'\d+', fname.split('.')[0])[0]
    idx.append(int(idx_num))
    
    image, _ = nrrd.read(ct_path['Image'])
    mask, _ = nrrd.read(ct_path['Mask'])
    
    image = (image - np.mean(image)) / (np.std(image) + 1e-12)    

    cropped = image * mask
    data = cropped.ravel()
    
    # Binning operation as conducted in PyRadiomics.
    minimum = min(data)
    maximum = max(data)

    low_32_bound = minimum - (minimum % width32)
    low_64_bound = minimum - (minimum % width64)
    low_128_bound = minimum - (minimum % width128)

    high_32_bound = maximum + 2 * width32
    high_64_bound = maximum + 2 * width64
    high_128_bound = maximum + 2 * width128

    bin_32_edges = np.arange(low_32_bound, high_32_bound, width32)
    bin_64_edges = np.arange(low_64_bound, high_64_bound, width64)
    bin_128_edges = np.arange(low_128_bound, high_128_bound, width128)
    
    gl_32bins[num] = bin_32_edges
    gl_64bins[num] = bin_64_edges
    gl_128bins[num] = bin_128_edges

In [ ]:
gl_32bins.shape, gl_64bins.shape, gl_128bins.shape

In [ ]:
# The average number of bins per GL discretization for the CURRENT images.
np.mean(gl_32bins), np.mean(gl_64bins), np.mean(gl_128bins)

In [ ]:
np.median(gl_32bins), np.median(gl_64bins), np.median(gl_128bins)

In [ ]:
np.std(gl_32bins), np.std(gl_64bins), np.std(gl_128bins)

In [ ]:
np.min(gl_32bins), np.min(gl_64bins), np.min(gl_128bins)

In [ ]:
np.max(gl_32bins), np.max(gl_64bins), np.max(gl_128bins)

# Data Prep

In [ ]:
y = np.squeeze(pd.read_csv(path_to_target, index_col=0).values)
X = pd.read_csv(path_to_features, index_col=0)
X.head()

In [ ]:
clinical_features = pd.read_csv('./../../data_source/to_analysis/clinical_params.csv', index_col=0)
pet_params = pd.read_csv('./../../data_source/to_analysis/pet_params.csv', index_col=0)

shape_features = X.filter(regex='shape')
    
clinical_features = clinical_features.loc[X.index, :]
pet_params = pet_params.loc[X.index, :]

shape_features.shape, clinical_features.shape, pet_params.shape

In [ ]:
CT_feats = X.filter(regex='CT')
PET_feats = X.filter(regex='PET')

CT_feats.shape, PET_feats.shape

In [ ]:
CT_fo = CT_feats.filter(regex='firstorder')
CT_fo.head()

In [ ]:
PET_fo = PET_feats.filter(regex='firstorder')
PET_fo.head()

In [ ]:
CT_text = CT_feats.drop(CT_fo.columns, axis=1)
CT_text.head()

In [ ]:
PET_text = PET_feats.drop(PET_fo.columns, axis=1)
# Skip PET parameter features.
pet_param_cols = [col for col in PET_text.columns if 'PETparam' in col]
PET_text.drop(pet_param_cols, axis=1, inplace=True)
PET_text.head()

# ICC & Hassan Transform of CT Texture Features

In [ ]:
# Apply Hassan transform and record which features are transformed.
gl_bins = [gl_32bins, gl_64bins, gl_128bins]
df_ct_icc = calc_icc(CT_text, gl_bins)
df_ct_icc.head()

In [ ]:
# Plotting original and transformed CT texture features.
plt.figure(figsize=(15, 10))
fig = sns.barplot(
    x=df_ct_icc.index, 
    y='Score', 
    hue='Kind', 
    data=df_ct_icc, 
    palette='muted',
)
plt.ylabel('Intraclass Correlation\nCoefficient', fontsize=20)
# Produces two sets of each label: one set for original, and one set for transformed 
# features.
labels = np.unique(prep_labels(hassan_transforms.keys()))
plt.xticks(
    np.arange(len(labels)), labels,
    rotation=45, ha='right', fontsize=17
)
plt.yticks(fontsize=17)

for patch_num, patch in enumerate(fig.patches):
    current_width = patch.get_width()
    diff = current_width - 0.3
    patch.set_width(0.3)
    # Recenter bars.
    patch.set_x(patch.get_x() + diff * 0.5)

plt.axhline(y=ICC_THRESH, linestyle='--', alpha=0.5, color='darkblue')
plt.legend(
    fontsize=17,
    title='Feature Definition:', title_fontsize=20,
    loc='upper center', 
    bbox_to_anchor=(0.5, 1.22),
    ncol=2, 
    fancybox=True, 
    shadow=True
)
plt.tight_layout()
plt.savefig('./../../figures/compressing_feature_space/orig_images_hassan_icc_ct_texture.png')

# ICC & Hassan Transform PET Texture Features

In [ ]:
gl_bins = [gl_32bins, gl_64bins, gl_128bins]
df_pet_icc = calc_icc(PET_text, gl_bins)

In [ ]:
# Plotting original and transformed CT texture features.
plt.figure(figsize=(15, 10))
fig = sns.barplot(
    x=df_pet_icc.index, 
    y='Score', 
    hue='Kind', 
    data=df_pet_icc, 
    palette='muted',
)
plt.ylabel('Intraclass Correlation Coefficient', fontsize=20)
# Produces two sets of each label: one set for original, and one set for transformed 
# featuresm.
labels = np.unique(prep_labels(hassan_transforms.keys(), pref='PET'))
plt.xticks(
    np.arange(len(labels)), labels,
    rotation=45, ha='right', fontsize=17
)
plt.yticks(fontsize=17)

for patch_num, patch in enumerate(fig.patches):
    current_width = patch.get_width()
    diff = current_width - 0.3
    patch.set_width(0.3)
    # Recenter bars.
    patch.set_x(patch.get_x() + diff * 0.5)

plt.axhline(y=ICC_THRESH, linestyle='--', alpha=0.5, color='darkblue')
plt.legend(
    fontsize=17,
    title='Feature Definition:', title_fontsize=18,
    loc='upper center', 
    bbox_to_anchor=(0.5, 1.25),
    ncol=2, 
    fancybox=True, 
    shadow=True
)
plt.tight_layout()
plt.savefig('./../../figures/compressing_feature_space/orig_images_hassan_icc_pet_texture.png')

# Remove Redundancy in PET and CT texture features

In [ ]:
def hassan_to_feat_space(X, to_transf, gl_bins, prefix):
    
    # Modified only the spcified features. Other features 
    # are retained in original version.
    X_red = X.copy()
    for cols_to_transf in to_transf:

        feats = X.filter(regex=cols_to_transf)
        tmp_feats = np.zeros_like(feats) 
        transform = hassan_transforms[cols_to_transf]
        
        for bin_num, (col, nbins) in enumerate(zip(feats.columns, gl_bins)):
            tmp_feats[:, bin_num] = transform(nbins, feats.loc[:, col].values)
            
        if icc(feats.values) < icc(tmp_feats):
            new_feat = np.mean(tmp_feats, axis=1)
            X_red.loc[:, f'{prefix}_{cols_to_transf}'] = new_feat
            X_red.drop(feats.columns, axis=1, inplace=True)
        else:
            print(f'Not transforming {cols_to_transf}')
        
    return X_red

In [ ]:
ct_transf_scores = df_ct_icc.loc[df_ct_icc.loc[:, 'Kind'] == 'Modified', 'Score']
ct_to_modify = (ct_transf_scores > ICC_THRESH).index.values

# Apply Hassan transform to features with ICC exceeding thresh. Drop redundant features.
CT_text_red = hassan_to_feat_space(CT_text, ct_to_modify, gl_bins, prefix='CT')
CT_text_red.head()

In [ ]:
pet_transf_scores = df_pet_icc.loc[df_pet_icc.loc[:, 'Kind'] == 'Modified', 'Score']
pet_to_modify = (pet_transf_scores > ICC_THRESH).index.values

# Apply Hassan transform to features with ICC exceeding thresh. Drop redundant features.
PET_text_red = hassan_to_feat_space(PET_text, pet_to_modify, gl_bins, prefix='PET')
PET_text_red.head()

# ICC of Remaining PET & CT Texture & First Order Features

The ICC may be used in further reduction of the feature space by calculating the ICC of each unique feature across image value discretizations, and remove those features with high correlations.

In [ ]:
def combine_correlated_features(X, corr, prefix):
    
    X_red = X.copy()
    for col in corr:
        feats = X.filter(regex=col)
        X_red.loc[:, f'{prefix}_{col}'] = np.mean(feats, axis=1)
        X_red.drop(feats.columns, axis=1, inplace=True)
        
    return X_red

In [ ]:
ct_fo_col_regexes = column_regexes(CT_fo)

ct_fo_icc = []
for ct_fo_regex in ct_fo_col_regexes:
    feats = CT_fo.filter(regex=ct_fo_regex)
    if len(feats.columns) == 3:
        ct_fo_icc.append(icc(feats.values))
    else:
        ct_fo_icc.append(0.0)
        
ct_fo_icc = np.array(ct_fo_icc)
ct_fo_to_mod = ct_fo_col_regexes[ct_fo_icc > ICC_THRESH]
CT_fo_red = combine_correlated_features(CT_fo, ct_fo_to_mod, prefix='CT')
CT_fo_red.index = X.index
CT_fo_red.shape

In [ ]:
ct_text_red_col_regexes = column_regexes(CT_text_red)

ct_text_red_icc = []
for ct_text_regex in ct_text_red_col_regexes:
    feats = CT_text_red.filter(regex=ct_text_regex)
    if len(feats.columns) == 3:
        ct_text_red_icc.append(icc(feats.values))
    else:
        ct_text_red_icc.append(0.0)
        
ct_text_red_icc = np.array(ct_text_red_icc)
ct_text_red_to_mod = ct_text_red_col_regexes[ct_text_red_icc > ICC_THRESH]
CT_text_red = combine_correlated_features(CT_text_red, ct_text_red_to_mod, prefix='CT')
CT_text_red.index = X.index
CT_text_red.shape

In [ ]:
pet_fo_col_regexes = column_regexes(PET_fo)

pet_fo_icc = []
for pet_fo_regex in pet_fo_col_regexes:
    feats = PET_fo.filter(regex=pet_fo_regex)
    if len(feats.columns) == 3:
        pet_fo_icc.append(icc(feats.values))
    else:
        pet_fo_icc.append(0.0)
    
pet_fo_icc = np.array(pet_fo_icc)
pet_fo_to_mod = pet_fo_col_regexes[pet_fo_icc > ICC_THRESH]
PET_fo_red = combine_correlated_features(PET_fo, pet_fo_to_mod, prefix='PET')
PET_fo_red.index = X.index
PET_fo_red.shape

In [ ]:
pet_text_red_col_regexes = column_regexes(PET_text_red)

pet_text_red_icc = []
for pet_text_regex in pet_text_red_col_regexes:
    feats = PET_text_red.filter(regex=pet_text_regex)
    if len(feats.columns) == 3:
        pet_text_red_icc.append(icc(feats.values))
    else:
        pet_text_red_icc.append(0.0)
        
pet_text_red_icc = np.array(pet_text_red_icc)
pet_text_red_to_mod = pet_text_red_col_regexes[pet_text_red_icc > ICC_THRESH]
PET_text_red = combine_correlated_features(PET_text_red, pet_text_red_to_mod, prefix='PET')
PET_text_red.index = X.index
PET_text_red.shape

In [ ]:
X_red = pd.concat(
    (
        #clinical_features, 
        shape_features, 
        CT_fo_red,
        CT_text_red, 
        PET_fo_red,
        PET_text_red, 
        pet_params, 
    ), 
    axis=1
)
X_red.head()

In [ ]:
X_red.to_csv(path_to_icc)
X_red.shape

# Dropping Highly Correlated Features

In [ ]:
# X_red: Removed featues with ICC > 0.8.
red_corr_matrix = X_red.corr(method='spearman').abs()
upper_red = red_corr_matrix.where(np.triu(np.ones(red_corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper_red.columns if any(upper_red[column] > 0.95)]
X_red_dropped = X_red.drop(to_drop, axis=1)

In [ ]:
# X_red_dropped: Removed featues with ICC > 0.8 and features with SCC > 0.95.
red_dropped_corr_matrix = X_red_dropped.corr(method='spearman').abs()
upper_red_dropped = red_dropped_corr_matrix.where(
    np.triu(np.ones(red_dropped_corr_matrix.shape), k=1).astype(np.bool)
)

In [ ]:
# Get largest correlation between two features.
red_max_feature_corr = []
for num, col in enumerate(upper_red.columns):
    if not np.isnan(max(upper_red[col])):
        red_max_feature_corr.append(max(upper_red[col]))

red_max_feature_corr = np.array(red_max_feature_corr)

red_dropped_max_feature_corr = []
for num, col in enumerate(upper_red_dropped.columns):
    if not np.isnan(max(upper_red_dropped[col])):
        red_dropped_max_feature_corr.append(max(upper_red_dropped[col]))

red_dropped_max_feature_corr = np.array(red_dropped_max_feature_corr)

In [ ]:
# The maximum Spearman's Correlation Coefficient a feature associates with
# another feature from the Hassan transformed feature matrix. That is, max SCC of features
# after the Hassan transforms.

red_feature_cats = np.array(to_feature_categories(upper_red.columns))
sorted_cats_idx = np.argsort(red_feature_cats)

to_drop = np.where(sorted_cats_idx == np.max(sorted_cats_idx))
sorted_cats_idx = np.delete(sorted_cats_idx, to_drop)
red_feature_cats = np.delete(red_feature_cats, to_drop)

plt.figure(figsize=(15, 7))
sns.scatterplot(
    np.arange(np.size(red_max_feature_corr)), 
    red_max_feature_corr[sorted_cats_idx],
    hue=red_feature_cats[sorted_cats_idx],
    palette=sns.color_palette('muted', len(np.unique(red_feature_cats))),
    s=50
)
plt.legend(
    fontsize=17, title_fontsize=20, title='Feature Category', loc='upper center', 
    bbox_to_anchor=(0.5, 0.3), ncol=3, fancybox=True, shadow=True
)
plt.ylabel("Maximum Associated\nSpearman's Correaltion Coefficient", fontsize=20)
plt.xlabel('Feature Indicator', fontsize=20)
plt.xticks(
    np.linspace(0, np.size(red_max_feature_corr), 6, dtype=int), 
    np.linspace(1, np.size(red_max_feature_corr), 6, dtype=int), 
    fontsize=17
)
plt.yticks(fontsize=17)

plt.savefig(
    './../../figures/compressing_feature_space/scc_after_icc_radiom_feats.png', 
    bbox_inches='tight',
    dpi=100, 
)

In [ ]:
# The maximum Spearman's Correlation Coefficient a feature associates with
# another feature from the Hassan transformed feature matrix including removal of 
# features with maximum SCC > 0.95.
red_dropped_feature_cats = np.array(to_feature_categories(upper_red_dropped.columns))
sorted_cats_idx = np.argsort(red_dropped_feature_cats)

to_drop = np.where(sorted_cats_idx == np.max(sorted_cats_idx))
sorted_cats_idx = np.delete(sorted_cats_idx, to_drop)
red_dropped_feature_cats = np.delete(red_dropped_feature_cats, to_drop)

plt.figure(figsize=(15, 7))
sns.scatterplot(
    np.arange(np.size(red_dropped_max_feature_corr)), 
    red_dropped_max_feature_corr[sorted_cats_idx],
    hue=red_dropped_feature_cats[sorted_cats_idx],
    palette=sns.color_palette('muted', len(np.unique(red_dropped_feature_cats))),
    s=50
)
plt.legend(
    fontsize=17, title_fontsize=20, title='Feature Category', loc='upper center', 
    bbox_to_anchor=(0.6, 0.3), ncol=3, fancybox=True, shadow=True
)
plt.ylabel("Maximum Associated\nSpearman's Correaltion Coefficient", fontsize=20)
plt.xlabel('Feature Indicator', fontsize=20)
plt.xticks(
    np.linspace(0, np.size(red_dropped_max_feature_corr), 6, dtype=int), 
    np.linspace(1, np.size(red_dropped_max_feature_corr), 6, dtype=int), 
    fontsize=17
)
plt.yticks(np.linspace(0.0, 1.0, 6), np.round(np.linspace(0.0, 1.0, 6), 1), fontsize=17)
plt.savefig(
    './../../figures/compressing_feature_space/orig_images_hassan_icc_scc_dropped_radiom_feats.png', 
    bbox_inches='tight',
    dpi=100, 
)

In [ ]:
# NB: Include clinical variables!
#X_red_dropped.to_csv(path_red_dropped)
#X_red_dropped.shape